In [1]:
import os
from typing import Any

import numpy as np
import pandas as pd
import tqdm

import algotrading_v40.structures.instrument_desc as sid
import algotrading_v40.utils.data_nav as udn
import algotrading_v40.utils.zerodha_data_cleaning as uzdc

data_dir = "/Users/chirayuagrawal/algotrading_v40/data/raw/indian_market"
parquet_files = []

for root, dirs, files in os.walk(data_dir):
  for file in files:
    if file.endswith(".parquet"):
      parquet_files.append(os.path.join(root, file))

raw_pqs = tuple(sorted(parquet_files))

In [2]:
def process(df: pd.DataFrame) -> tuple[pd.DataFrame, dict[str, Any]]:
  df = df[df["date"].dt.year >= 2016]
  fix_unusual_bars_result = uzdc.fix_unusual_bars(df)
  df = fix_unusual_bars_result.df
  df = uzdc.set_index_to_bar_close_timestamp(df)
  drop_non_standard_indian_trading_hours_result = (
    uzdc.drop_non_standard_indian_trading_hours(df)
  )
  df = drop_non_standard_indian_trading_hours_result.df
  fix_high_low_values_result = uzdc.fix_high_low_values(df)
  df = fix_high_low_values_result.df
  count_bars_per_trading_day_result = uzdc.count_bars_per_trading_day(df)
  quality_result = uzdc.analyse_numeric_columns_quality(df)
  for col in ["open", "high", "low", "close", "volume"]:
    if quality_result[col].n_bad_values > 0:
      raise ValueError(f"Column {col} has bad values")
    if quality_result[col].n_negatives > 0:
      raise ValueError(f"Column {col} has negatives")
    if col != "volume" and quality_result[col].n_zeros > 0:  # volume can have zeros
      raise ValueError(f"Column {col} has zeros")

  df_post_2022 = df[df.index.year >= 2022].copy()
  quality_result_post_2022 = uzdc.analyse_numeric_columns_quality(df_post_2022)

  return df, {
    "n_unusual_bars_dropped": fix_unusual_bars_result.n_dropped,
    "n_unusual_bars_fixed": fix_unusual_bars_result.n_date_fixed,
    "n_non_standard_indian_trading_hours_dropped": drop_non_standard_indian_trading_hours_result.n_dropped,
    "n_high_fixed": fix_high_low_values_result.n_high_fixed,
    "n_low_fixed": fix_high_low_values_result.n_low_fixed,
    "percentage_dates_with_less_than_375_bars": np.round(
      100 * count_bars_per_trading_day_result.fraction_dates_with_less_than_375_bars, 3
    ),
    "n_dates": count_bars_per_trading_day_result.n_dates,
    "n_dates_with_less_than_375_bars": count_bars_per_trading_day_result.n_dates_with_less_than_375_bars,
    "percentage_zero_volume_bars": np.round(
      100 * quality_result["volume"].n_zeros / len(df), 3
    ),
    "percentage_zero_volume_bars_post_2022": np.round(
      100 * quality_result_post_2022["volume"].n_zeros / len(df_post_2022), 3
    ),
    "n_zero_volume_bars": quality_result["volume"].n_zeros,
    "n_zero_volume_bars_post_2022": quality_result_post_2022["volume"].n_zeros,
    "n_bars": len(df),
    "first_date": df.index.min().date(),
    "last_date": df.index.max().date(),
  }

In [3]:
rows = []
for raw_pq in tqdm.tqdm(raw_pqs):
  instrument_desc = udn.get_instrument_desc_from_path(raw_pq)
  df = pd.read_parquet(raw_pq)
  dfp, row = process(df)
  del df
  path = udn.get_cleaned_path_from_instrument_desc(instrument_desc)
  os.makedirs(os.path.dirname(path), exist_ok=True)
  dfp.to_parquet(path)
  row["symbol"] = instrument_desc.symbol
  row["type"] = "equity" if isinstance(instrument_desc, sid.EquityDesc) else "index"
  rows.append(row)

df_inventory_cleaned = pd.DataFrame(rows)
df_inventory_cleaned

100%|██████████| 55/55 [00:46<00:00,  1.17it/s]


,n_unusual_bars_dropped,n_unusual_bars_fixed,n_non_standard_indian_trading_hours_dropped,n_high_fixed,n_low_fixed,percentage_dates_with_less_than_375_bars,n_dates,n_dates_with_less_than_375_bars,percentage_zero_volume_bars,percentage_zero_volume_bars_post_2022,n_zero_volume_bars,n_zero_volume_bars_post_2022,n_bars,first_date,last_date,symbol,type
0,0,0,616,0,0,0.934,2356,22,0.235,0.002,2075,7,882320,2016-01-04,2025-07-18,ADANIENT,equity
1,0,0,615,0,0,0.806,2356,19,0.006,0.002,53,6,882379,2016-01-04,2025-07-18,ADANIPORTS,equity
2,0,0,615,0,0,0.849,2356,20,0.441,0.005,3892,18,882378,2016-01-04,2025-07-18,APOLLOHOSP,equity
3,0,0,615,0,0,0.806,2356,19,0.009,0.002,77,7,882379,2016-01-04,2025-07-18,ASIANPAINT,equity
4,0,0,616,0,0,0.806,2356,19,0.001,0.002,8,6,882379,2016-01-04,2025-07-18,AXISBANK,equity
5,0,0,616,0,0,0.849,2356,20,0.111,0.006,983,21,882376,2016-01-04,2025-07-18,BAJAJ-AUTO,equity
6,0,0,616,0,0,1.019,2356,24,0.919,0.002,8107,6,882370,2016-01-04,2025-07-18,BAJAJFINSV,equity
7,0,0,541,0,0,0.849,2356,20,0.090,0.002,797,5,882377,2016-01-04,2025-07-18,BAJFINANCE,equity
8,0,0,616,0,0,0.806,2356,19,0.150,0.002,1325,6,882378,2016-01-04,2025-07-18,BEL,equity
9,0,0,616,1,0,0.806,2356,19,0.008,0.002,75,5,882378,2016-01-04,2025-07-18,BHARTIARTL,equity


In [4]:
df_inventory_raw = pd.read_csv(
  "/Users/chirayuagrawal/algotrading_v40/data/raw/indian_market/inventory.csv",
  index_col=0,
)
raw_sym_type = sorted(
  [tuple(row) for row in df_inventory_raw[["symbol", "type"]].values.tolist()]
)
cleaned_sym_type = sorted(
  [tuple(row) for row in df_inventory_cleaned[["symbol", "type"]].values.tolist()]
)

if raw_sym_type != cleaned_sym_type:
  raise ValueError("raw_sym_type != cleaned_sym_type")

if len(set(raw_sym_type)) != len(raw_sym_type):
  raise ValueError("raw_sym_type has duplicates")
# df_inventory_raw and df_inventory_cleaned can be safely merged on symbol and type

In [5]:
dfm = (
  df_inventory_raw[["company_name", "industry", "symbol", "listing_date", "type"]]
  .merge(
    df_inventory_cleaned,
    on=["symbol", "type"],
    how="left",
  )
  .sort_values(by=["type", "industry", "symbol"])
  .reset_index(drop=True)
)
dfm

,company_name,industry,symbol,listing_date,type,n_unusual_bars_dropped,n_unusual_bars_fixed,n_non_standard_indian_trading_hours_dropped,n_high_fixed,n_low_fixed,percentage_dates_with_less_than_375_bars,n_dates,n_dates_with_less_than_375_bars,percentage_zero_volume_bars,percentage_zero_volume_bars_post_2022,n_zero_volume_bars,n_zero_volume_bars_post_2022,n_bars,first_date,last_date
0,Bajaj Auto Ltd.,Automobile and Auto Components,BAJAJ-AUTO,2008-05-26,equity,0,0,616,0,0,0.849,2356,20,0.111,0.006,983,21,882376,2016-01-04,2025-07-18
1,Eicher Motors Ltd.,Automobile and Auto Components,EICHERMOT,2004-09-07,equity,0,0,616,0,0,0.849,2356,20,0.123,0.003,1082,11,882370,2016-01-04,2025-07-18
2,Hero MotoCorp Ltd.,Automobile and Auto Components,HEROMOTOCO,2003-04-11,equity,0,0,615,0,0,0.849,2356,20,0.027,0.003,238,10,882370,2016-01-04,2025-07-18
3,Mahindra & Mahindra Ltd.,Automobile and Auto Components,M&M,1996-01-03,equity,0,0,616,0,0,0.849,2356,20,0.007,0.002,60,5,882376,2016-01-04,2025-07-18
4,Maruti Suzuki India Ltd.,Automobile and Auto Components,MARUTI,2003-07-09,equity,0,0,616,0,0,0.891,2356,21,0.001,0.002,12,7,882374,2016-01-04,2025-07-18
5,Tata Motors Ltd.,Automobile and Auto Components,TATAMOTORS,1998-07-22,equity,0,0,616,0,0,0.849,2356,20,0.001,0.002,9,8,882372,2016-01-04,2025-07-18
6,Bharat Electronics Ltd.,Capital Goods,BEL,2000-07-19,equity,0,0,616,0,0,0.806,2356,19,0.150,0.002,1325,6,882378,2016-01-04,2025-07-18
7,Larsen & Toubro Ltd.,Construction,LT,2004-06-23,equity,0,0,615,0,0,0.849,2356,20,0.001,0.002,8,5,882375,2016-01-04,2025-07-18
8,Grasim Industries Ltd.,Construction Materials,GRASIM,1995-05-10,equity,0,0,616,0,0,0.891,2356,21,0.530,0.019,4674,63,882341,2016-01-04,2025-07-18
9,UltraTech Cement Ltd.,Construction Materials,ULTRACEMCO,2004-08-24,equity,0,0,616,0,0,0.849,2356,20,0.051,0.005,452,15,882372,2016-01-04,2025-07-18


In [6]:
dfm = dfm[
  [
    "company_name",
    "industry",
    "symbol",
    "listing_date",
    "type",
    "first_date",
    "last_date",
    "n_unusual_bars_dropped",
    "n_unusual_bars_fixed",
    "n_non_standard_indian_trading_hours_dropped",
    "n_high_fixed",
    "n_low_fixed",
    "percentage_dates_with_less_than_375_bars",
    "n_dates",
    "n_dates_with_less_than_375_bars",
    "percentage_zero_volume_bars",
    "percentage_zero_volume_bars_post_2022",
    "n_zero_volume_bars",
    "n_zero_volume_bars_post_2022",
    "n_bars",
  ]
]
dfm

,company_name,industry,symbol,listing_date,type,first_date,last_date,n_unusual_bars_dropped,n_unusual_bars_fixed,n_non_standard_indian_trading_hours_dropped,n_high_fixed,n_low_fixed,percentage_dates_with_less_than_375_bars,n_dates,n_dates_with_less_than_375_bars,percentage_zero_volume_bars,percentage_zero_volume_bars_post_2022,n_zero_volume_bars,n_zero_volume_bars_post_2022,n_bars
0,Bajaj Auto Ltd.,Automobile and Auto Components,BAJAJ-AUTO,2008-05-26,equity,2016-01-04,2025-07-18,0,0,616,0,0,0.849,2356,20,0.111,0.006,983,21,882376
1,Eicher Motors Ltd.,Automobile and Auto Components,EICHERMOT,2004-09-07,equity,2016-01-04,2025-07-18,0,0,616,0,0,0.849,2356,20,0.123,0.003,1082,11,882370
2,Hero MotoCorp Ltd.,Automobile and Auto Components,HEROMOTOCO,2003-04-11,equity,2016-01-04,2025-07-18,0,0,615,0,0,0.849,2356,20,0.027,0.003,238,10,882370
3,Mahindra & Mahindra Ltd.,Automobile and Auto Components,M&M,1996-01-03,equity,2016-01-04,2025-07-18,0,0,616,0,0,0.849,2356,20,0.007,0.002,60,5,882376
4,Maruti Suzuki India Ltd.,Automobile and Auto Components,MARUTI,2003-07-09,equity,2016-01-04,2025-07-18,0,0,616,0,0,0.891,2356,21,0.001,0.002,12,7,882374
5,Tata Motors Ltd.,Automobile and Auto Components,TATAMOTORS,1998-07-22,equity,2016-01-04,2025-07-18,0,0,616,0,0,0.849,2356,20,0.001,0.002,9,8,882372
6,Bharat Electronics Ltd.,Capital Goods,BEL,2000-07-19,equity,2016-01-04,2025-07-18,0,0,616,0,0,0.806,2356,19,0.150,0.002,1325,6,882378
7,Larsen & Toubro Ltd.,Construction,LT,2004-06-23,equity,2016-01-04,2025-07-18,0,0,615,0,0,0.849,2356,20,0.001,0.002,8,5,882375
8,Grasim Industries Ltd.,Construction Materials,GRASIM,1995-05-10,equity,2016-01-04,2025-07-18,0,0,616,0,0,0.891,2356,21,0.530,0.019,4674,63,882341
9,UltraTech Cement Ltd.,Construction Materials,ULTRACEMCO,2004-08-24,equity,2016-01-04,2025-07-18,0,0,616,0,0,0.849,2356,20,0.051,0.005,452,15,882372


In [7]:
dfm.to_csv(
  "/Users/chirayuagrawal/algotrading_v40/data/cleaned/indian_market/inventory.csv",
  index=False,
)